pip install selenium

In [57]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()

try:

    driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

    select_element = driver.find_element(By.ID,'uregi')
    select_object = Select(select_element)

    select_object.select_by_index(5)

    search_button = driver.find_element(By.ID, "ok")
    search_button.click()


    #Pegando o arquivo e as infos de cada linha

    n = 2

    lista_info = []

    for i in range(1,6):
        lista_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
        print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)


    print(lista_info)

    
    print ("E a data?")

    #município = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[1]").text
    #regional_ensino = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[2]").text
    #meta = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[3]").text
    #cliente = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[4]").text
    
    data = driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[1]/td[5]")
    print(data)
    
    data1 = driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[7]/p[2]")
    print(data.get_attribute("number"))
    print(data.get_attribute("date"))
    print(data.get_attribute("value"))
    print(data.get_attribute("text"))
    print(data.get_attribute("name"))
    print(data.get_property("number"))
    print(data.get_property("date"))
    print(data.get_property("text"))
    print(data.get_property("name"))
    print(data.id)
    print(data.text)
    
    
finally:
    driver.quit()


Rodeiro
Ubá
Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado
José Marcio de Mendonça

['Rodeiro', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'José Marcio de Mendonça', '']
E a data?
<selenium.webdriver.remote.webelement.WebElement (session="9fdbe52546fe5887b24fd77311a3f0c1", element="4a2af696-8336-4443-a640-2cb5497f6d73")>
None
None
None
None
None
None
None
None
None
4a2af696-8336-4443-a640-2cb5497f6d73



In [3]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()

try:

    driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

    select_element = driver.find_element(By.ID,'uregi')
    select_object = Select(select_element)

    select_object.select_by_index(5) # Posição na lista de seleção

    search_button = driver.find_element(By.ID, "ok")
    search_button.click()


    #Pegando o arquivo e as infos de cada linha
    

    """
    n = 2

    lista_info = []

    for i in range(1,6):
        lista_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
        print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)


    print(lista_info)
    """
   
    # Com essa porcaria consegui pegar todos os dados de uma vez
    elements = driver.find_elements(By.TAG_NAME, 'tr')
    i = 0
    for e in elements:
        print(e.text)
        #driver.execute_script("window.scrollTo(0, 300)")
        #driver.execute_script("rola = rola + 1000;")
        #driver.execute_script("window.scrollTo({top: rola,behavior: 'smooth'});")
        #objeto = driver.find_element(By.CLASS_NAME, "textoCentral")
        #objeto.sendKeys(Keys.DOWN)
        i += e.rect['height']
        driver.execute_script(f"window.scrollTo(0,{i})")

finally:
    pass
    #driver.quit()
  

Município Regional Ensino Meta Cliente Data Documentos Enviado SEI
Rodeiro Ubá Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado José Marcio de Mendonça 23/11/2021
09/02/2022
Rodeiro Ubá Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Edvar Nogueira Junior 24/11/2021
Rodeiro Ubá Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Evaldo José Nogueira 02/12/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Marcio Luiz Porto Pereira 16/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Bráz Pereira Dias 16/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Genésio Ferreira do Nascimento 16/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Jaime de Oliveira Dutra 17/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Nelci

Rochedo de Minas Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Anderson das Graças Filgueiras 06/01/2022
Rochedo de Minas Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Camilo de Lelis Detoni 13/01/2022
Rochedo de Minas Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Geomar de Paula Araujo 26/01/2022
Rochedo de Minas Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Paulo Roberto da Cunha 06/01/2022
Argirita Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Bernardo Iennaco de Moraes 22/11/2021
Argirita Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Genuina Moreira de Paula 22/11/2021
Argirita Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Vinicios Campos Guedes Silva 29/11/2021
Argirita Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Bernardo Iennaco de Moraes 29/11/2021
Argirita Leopold

Cataguases Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Nadir das Graças Menezes de Souza Lima,Nadir das Graças Menezes de Souza Lima 18/11/2021
Senador Cortes Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Selmo Chermont da Costa 17/12/2021
Senador Cortes Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Marlon Guerra Muniz 20/12/2021
Senador Cortes Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado João Batista Cazarim de Almeida 04/01/2022
Senador Cortes Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Divino de Freitas Filho 10/01/2022
Senador Cortes Juiz de Fora Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Marlon Guerra Muniz 14/01/2022
Piraúba Ubá Projeto 2 - Meta 2.1 - Reunião de Mobilização e Divulgação Maria Selma Barbosa Lima,Walney Rodrigues Martins,Adilson Lino Nepomuceno,Eduardo Guzela Pinheiro,Christiano Oliveira Duarte,Dirceu Amara

In [37]:
from selenium import webdriver

driver = webdriver.Chrome()

# Navigate to url
driver.get("https://www.example.com")

    # Returns height, width, x and y coordinates referenced element
res = driver.find_element(By.CSS_SELECTOR, "h1").rect

print(res)

{'height': 43, 'width': 600, 'x': 217, 'y': 133.4375}


In [30]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time



# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

select_element = driver.find_element(By.ID,'uregi')
select_object = Select(select_element)

select_object.select_by_index(5)

search_button = driver.find_element(By.ID, "ok")
search_button.click()


#Pegando o arquivo e as infos de cada linha e salvando em um dicionário

list_info = []
linha = 2
scroll = 0

while True:
    
    #WebDriverWait(driver).until(lambda d: d.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]))
    #expected_conditions.presence_of_element_located(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}])
    
    #Pegando as informações de cada linha e colocando em uma lista
    for i in range(1,6):
        list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]").text)
        #print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
    print(list_info)
    
    
    
    
    
    #Clicando na imagem do arquivo
    driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]").click()
    
    #WebDriverWait(driver, 10).until(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]"))
    
    #WebDriverWait(driver, 10).until(lambda d: d.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]"))
    
    #Tentando arrastar pro lado pra corrigir o problema do IndexError: list index out of range
    #driver.execute_script("window.scrollTo(0, document.body.scrollWeight);")
    #ActionChains(driver).click_and_hold(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]")).move_by_offset(2000 , 0).release().perform()
    
    
    try:
        driver.switch_to.window(driver.window_handles[1])
    
    except:
        #Arrastar pra direita
        #driver.find_element(By.TAG_NAME, f"html").send_keys(Keys.ARROW_RIGHT)
        
        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        print(scroll)
        driver.execute_script(f"window.scrollTo(0,{scroll})") #x,y
        linha = linha + 1
        list_info.clear()
    
    else:
                
        driver.find_element(By.LINK_TEXT, "Download").click()

        #Close the tab or window
        driver.close()

        #Switch back to the old tab or window
        driver.switch_to.window(original_window)

        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        driver.execute_script(f"window.scrollTo(500,{scroll})")

        linha = linha + 1
        list_info.clear()

        



['Rodeiro', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'José Marcio de Mendonça', '23/11/2021']
['Rodeiro', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Edvar Nogueira Junior', '24/11/2021']
['Rodeiro', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Evaldo José Nogueira', '02/12/2021']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Marcio Luiz Porto Pereira', '16/11/2021']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Bráz Pereira Dias', '16/11/2021']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Genésio Ferreira do Nascimento', '16/11/2021']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Jaime de Oliveira Dutra', '17/11/2021']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - A

['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Bráz Pereira Dias', '17/01/2022']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Marcio Luiz Porto Pereira', '18/01/2022']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Rodrigo Zignago', '13/01/2022']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Gessi Rocha Ferreira', '22/11/2021']
['Itamarati de Minas', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Jaime de Oliveira Dutra', '22/11/2021']
['São João Nepomuceno', 'Juiz de Fora', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Jeferson Aparecido Dornelas Reis,Jeferson Aparecido Dornelas Reis', '17/12/2021']
['Astolfo Dutra', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'José Antônio de Castro', '07

['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Samuel José de Souza', '18/01/2022']
['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Clévio Braz', '18/01/2022']
['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Claudinei Rodrigues da Silva', '13/01/2022']
['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Ademir Rodrigues da Silva', '13/01/2022']
['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Vitor Marques do Amaral', '04/01/2022']
['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Marcelo Marques Amaral', '04/01/2022']
['Tocantins', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Otacilia Neves Moreira de Freitas,Otacilia Neves Moreira de Freitas', '27/01/2022']
['Argirita', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Me

['Cataguases', 'Leopoldina', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Gilberto Severino de Castro,Gilberto Severino de Castro', '21/12/2021']
['Santana de Cataguases', 'Muriaé', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'Francisco de Assis do Carmo', '08/12/2021']
['São João Nepomuceno', 'Juiz de Fora', 'Projeto 2 - Meta 2.1 - Reunião de Mobilização e Divulgação', 'Karine de Castro Dias Duque Fernandes,Maria do Rosário de Lima Souza,Marina Pereira Gomes Marchiore,Natália Maria Machado Matos,Paulo Afonso Nogueira Junior,Sara Kenya F Caquito,Vanderley Ribeiro Duque,Vanessa Clemente Pereira,Adriana Cardoso de Mattos,Adriana de Oliveira Siqueira,Jeferson Aparecido Dornelas Reis,Sebastião Heleno Candido da Silva,Antonio Márcio Novato do Carmo,José Maria Detoni Júnior,Rosimeri de Mendonça Torres,Gustavo Rodrigues Andrade,Luciano Felipe de Mendonça,Clener da Silva Cúrcio,Flávio de Paula Fontes', '14/02/2022']
8760
['Santana de Cataguases', 'Muriaé

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/section/div/div[1]/table/tbody/tr[176]/td[1]"}
  (Session info: chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C47AC3+2587331]
	Ordinal0 [0x00BDADD1+2141649]
	Ordinal0 [0x00AD3BB8+1063864]
	Ordinal0 [0x00B001CE+1245646]
	Ordinal0 [0x00B003CB+1246155]
	Ordinal0 [0x00B2A612+1418770]
	Ordinal0 [0x00B186D4+1345236]
	Ordinal0 [0x00B28A0A+1411594]
	Ordinal0 [0x00B184A6+1344678]
	Ordinal0 [0x00AF53F6+1201142]
	Ordinal0 [0x00AF62E6+1204966]
	GetHandleVerifier [0x00DEDF22+1680738]
	GetHandleVerifier [0x00EA0DBC+2413564]
	GetHandleVerifier [0x00CDD151+563089]
	GetHandleVerifier [0x00CDBF13+558419]
	Ordinal0 [0x00BE081E+2164766]
	Ordinal0 [0x00BE5508+2184456]
	Ordinal0 [0x00BE5650+2184784]
	Ordinal0 [0x00BEF5BC+2225596]
	BaseThreadInitThunk [0x77866359+25]
	RtlGetAppContainerNamedObjectPath [0x77A07C24+228]
	RtlGetAppContainerNamedObjectPath [0x77A07BF4+180]
